In [1]:
!pip -q install qiskit ipywidgets

     |████████████████████████████████| 6.0MB 6.0MB/s 
     |████████████████████████████████| 17.9MB 318kB/s 
     |████████████████████████████████| 204kB 73.6MB/s 
     |████████████████████████████████| 215kB 66.1MB/s 
     |████████████████████████████████| 2.1MB 60.3MB/s 
     |████████████████████████████████| 1.0MB 57.3MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 194kB 58.0MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
     |████████████████████████████████| 614kB 57.6MB/s 
     |████████████████████████████████| 3.2MB 51.2MB/s 
     |████████████████████████████████| 5.5MB 62.5MB/s 


# DQC1 QISKIT


In [8]:
import numpy as np
import itertools
shots = 100000

from qiskit import(
  QuantumCircuit,
  execute,
  Aer)
from qiskit.visualization import plot_histogram
simulator = Aer.get_backend('qasm_simulator')

def simulate_dqc1_qiskit(ensemble_qubits, angles_d, img = False):

    m_counts = 0.0
    bin_nums = [list(i) for i in itertools.product([0, 1], repeat=ensemble_qubits)]
    for bin_num in bin_nums:

        circuit = QuantumCircuit(ensemble_qubits + 1, 1)
        circuit.h(0)
        
        for j in range(ensemble_qubits):
            if 1 == bin_num[j]:
                circuit.x(j + 1)

        for i in range(ensemble_qubits):
            circuit.cu(angles_d[i]['theta'], angles_d[i]['phi'], angles_d[i]['lambda'], 0, 0, i + 1)

        if img:
            circuit.sdg(0)
        circuit.h(0) 

        circuit.measure([0], [0])

        job = execute(circuit, simulator, shots=shots)
        result = job.result()
        counts = result.get_counts(circuit)
        m_counts += counts["0"] - counts["1"]

    return (0.5 ** ensemble_qubits) * m_counts / shots


In [9]:
ensemble_qubits = 3
angles_d = []

for i in range(ensemble_qubits):
    theta, phi, lam = np.pi * np.random.rand(), np.pi * np.random.rand(), np.pi * 2* np.random.rand()
    angles_d.append({ "theta": theta, "phi":phi, "lambda":  lam})

    print('Testing Theta{} = {} Phi{} = {} Lam{} = {}'.format(i,'{0:.6f}'.format(theta), i, '{0:.6f}'.format(phi), i, '{0:.6f}'.format(lam)))

real1, img1 = simulate_dqc1_qiskit(ensemble_qubits, angles_d, img = False), simulate_dqc1_qiskit(ensemble_qubits, angles_d, img = True)
print('Real', real1.real, 'Img', img1.real,' - Qiskit Simulator')


Testing Theta0 = 1.015725 Phi0 = 1.716767 Lam0 = 6.052044
Testing Theta1 = 0.660778 Phi1 = 2.963584 Lam1 = 6.187359
Testing Theta2 = 2.746233 Phi2 = 1.856983 Lam2 = 4.599187
Real -0.011485 Img 0.01126  - Qiskit Simulator
